### 설정 + 필요한 모델 다운로드

In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 4.6 MB/s 
     |████████████████████████████████| 448 kB 70.9 MB/s 


In [2]:
!pip install git+https://github.com/SKT-AI/KoBART #egg=kobart

  Cloning https://github.com/SKT-AI/KoBART to /tmp/pip-req-build-zvprajqg
  Running command git clone -q https://github.com/SKT-AI/KoBART /tmp/pip-req-build-zvprajqg
     |████████████████████████████████| 132 kB 4.1 MB/s 
     |████████████████████████████████| 814 kB 32.0 MB/s 
     |████████████████████████████████| 776.8 MB 17 kB/s 
     |████████████████████████████████| 1.9 MB 24.0 MB/s 
     |████████████████████████████████| 596 kB 74.0 MB/s 
     |████████████████████████████████| 134 kB 100.5 MB/s 
     |████████████████████████████████| 829 kB 65.1 MB/s 
     |████████████████████████████████| 895 kB 74.3 MB/s 
     |████████████████████████████████| 3.3 MB 22.8 MB/s 
     |████████████████████████████████| 1.1 MB 57.0 MB/s 
     |████████████████████████████████| 271 kB 89.3 MB/s 
     |████████████████████████████████| 144 kB 61.5 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 79 kB 10.8 MB/s 
     |███████████████████

In [3]:
!pip install transformers

In [4]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 4.3 MB/s 
     |████████████████████████████████| 144 kB 78.1 MB/s 
     |████████████████████████████████| 181 kB 94.7 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=66708e5fae41263019141b69e125d470798f72876557b3249d3237863b2e89ac
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [5]:
import torch
import os 
import sys
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import wandb

from google.colab import drive
import re
from konlpy.tag import Okt
import requests
from bs4 import BeautifulSoup

from itertools import combinations
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader,random_split

from kobart import get_pytorch_kobart_model, get_kobart_tokenizer
from transformers import AutoModel, AutoTokenizer, BartModel

In [6]:
# seed
seed = 7777
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla P100-PCIE-16GB
cuda


### 원본 데이터 불러오기

In [7]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
df = pd.read_csv("/content/drive/MyDrive/NLP/sports_news_data - sports_news_data.csv")

In [9]:
df[df['CONTENT'].isnull() == True]

,TITLE,CONTENT,PUBLISH_DT
4137,GOAL50 2021 투표하기,NaN,2021-11-02
5475,"[GOAL LIVE] '오직 익수' 안익수가 생각하는 팬의 의미 ""상당히 두려운 존재""",NaN,2021-12-04


In [10]:
# 결측치 확인
df.isnull().sum()

TITLE         0
CONTENT       2
PUBLISH_DT    0
dtype: int64

In [11]:
# 결측치 제거
df = df.dropna()
df.isnull().sum()

TITLE         0
CONTENT       0
PUBLISH_DT    0
dtype: int64

In [12]:
# 중복값 제거(CONTENT만 처리하므로 TITLE은 두기로 한다.)
idx = df['CONTENT'].drop_duplicates().index
df = df.loc[idx]
df.reset_index(drop=True, inplace = True)
print(df.shape) ; df.head(5)

(9050, 3)


,TITLE,CONTENT,PUBLISH_DT
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,[스포탈코리아] 유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 ...,2021-07-15
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",[스포탈코리아] 유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤...,2021-07-15
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,[스포탈코리아] 새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\n\...,2021-07-15
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",[스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠...,2021-07-15
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",[스포탈코리아] FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽...,2021-07-15


### 전처리
- 중복 및 결측치 제거
- 크롤링 상에서 생긴 쓸모 없는 문구 처리


In [ ]:
# 타이틀(안 쓸시 코멘트 처리 할 것)
def title_cleansing(x):
    new_string = re.sub(r'\([^)]*\)|\[[^)]*\]|\<[^)]*\>', '', x)
    new_string = re.sub(r"[^가-힣a-zA-Z0-9一-龥. ]","",new_string)
    
    return new_string

# 본문
def content_cleansing(string):
    try:
        if '스포탈코리아' in string:
            cleanr =re.compile('<.*?>')
            cleantext = re.sub(cleanr, '', string).replace("&nbsp;", "").replace('\n',"").replace('\t',"").replace('\xa0', "")


            new_string = re.sub(r'사진=.+$', '', cleantext)
            new_string = re.sub(r'.+.기자=', '', new_string)    
            new_string = re.sub(r'\([^)]*\)|\[[^)]*\]|\<[^)]*\>', '', new_string)
            new_string = re.sub(r"[^가-힣a-zA-Z0-9一-龥. ]","",new_string)

            return new_string.strip()

        elif '골닷컴' in string:   ### +++ 기자 이름
            string = re.sub(r'(?<=\<hr style="color: #00ac77; width: 100%; background: #00ac77; border: 0; height: 2px;" /><span><strong>)(.*?)(?=<\/span>)', '', string)

            cleanr =re.compile('<.*?>')
            cleantext = re.sub(cleanr, '', string).replace("&nbsp;", "").replace('\n',"").replace('\t',"").replace('\xa0', "")

            new_string = re.sub(r'\([^)]*\)|\[[^)]*\]|\<[^)]*\>', '', cleantext)
            new_string = re.sub(r"[^가-힣a-zA-Z0-9一-龥. ]","", new_string)

            return new_string.strip()

        else:
            cleanr =re.compile('<.*?>')
            cleantext = re.sub(cleanr, '', string).replace("&nbsp;", "").replace('\n',"").replace('\t',"").replace('\xa0', "")

            new_string = re.sub(r'\([^)]*\)|\[[^)]*\]|\<[^)]*\>', '', cleantext)
            new_string = re.sub(r"[^가-힣a-zA-Z0-9一-龥. ]","", new_string)
            
            return new_string.strip()
        
    except:
        return "문제"

In [ ]:
df['cleaned_TITLE'] = df['TITLE'].progress_apply(lambda x: title_cleansing(x))
df['cleaned_CONTENT'] = df['CONTENT'].progress_apply(lambda x: content_cleansing(x))

100%|██████████| 9050/9050 [00:30<00:00, 295.97it/s]


In [ ]:
# 데이터 전처리 후 중복값 제거
idx = df['cleaned_CONTENT'].drop_duplicates().index
df = df.loc[idx]
df.reset_index(drop=True, inplace = True)
print(df.shape) ; df.head(5)

(8998, 5)


,TITLE,CONTENT,PUBLISH_DT,cleaned_TITLE,cleaned_CONTENT
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,[스포탈코리아] 유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 ...,2021-07-15,스털링 다이빙 논란 종결 오른쪽 다리 접촉 있었잖아,유럽축구연맹 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크전에 나온 ...
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",[스포탈코리아] 유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤...,2021-07-15,디 마리아 없다 유로X코파 베스트11 이탈리아만 7명,유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 모습일까.지난달...
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,[스포탈코리아] 새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\n\...,2021-07-15,슈퍼컴퓨터 예측 맨시티 우승맨유 4위 토트넘은 6위,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.영국 매체 스포츠 바이...
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",[스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠...,2021-07-15,이재성 완벽한 프로 마인츠서 성공할 것 킬 디렉터의 애정 듬뿍 응원,홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다.이재성은 20...
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",[스포탈코리아] FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽...,2021-07-15,홈킷과 딴판 바르사 팬들 NEW 어웨이 셔츠 호평 가장 좋아하는 색,FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽다는 반응이다.바...


In [ ]:
df_new = df[['cleaned_TITLE', 'cleaned_CONTENT']]
df_new.rename(columns={'cleaned_CONTENT': 'CONTENT', 'cleaned_TITLE':'TITLE'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
df_new

,TITLE,CONTENT
0,스털링 다이빙 논란 종결 오른쪽 다리 접촉 있었잖아,유럽축구연맹 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크전에 나온 ...
1,디 마리아 없다 유로X코파 베스트11 이탈리아만 7명,유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 모습일까.지난달...
2,슈퍼컴퓨터 예측 맨시티 우승맨유 4위 토트넘은 6위,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.영국 매체 스포츠 바이...
3,이재성 완벽한 프로 마인츠서 성공할 것 킬 디렉터의 애정 듬뿍 응원,홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다.이재성은 20...
4,홈킷과 딴판 바르사 팬들 NEW 어웨이 셔츠 호평 가장 좋아하는 색,FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽다는 반응이다.바...
...,...,...
8993,이제 홈팬 야유 받는 먹튀 선수 차비 감독 조차 그만 해라,FC바르셀로나 팬들에게 우스망 뎀벨레는 밉상이 되어버렸다. 바르사는 7일 오전 0시...
8994,성남 만 17세 유스 김지수와 준프로 계약,성남FC가 만17세 2004년생 수비수 김지수와 준프로 계약을 체결했다. 김지수는 ...
8995,오베르마스 아약스서 쫓겨난다...여성 동료들에게 부적절한 메시지,레전드 마르크 오베르마스가 아약스에서 쫓겨났다. 이유는 굉장히 굴욕적이었다.아약스는...
8996,바르사 차비 감독 트라오레 데뷔전 활약에 깜짝...몸이 야수 같아,FC 바르셀로나 차비 에르난데스 감독이 데뷔전을 치른 아다마 트라오레를 극찬했다.바...


In [ ]:
df_new['listed_CONTENT'] = df_new['CONTENT'].progress_apply(lambda x: x.split('.'))

100%|██████████| 8998/8998 [00:00<00:00, 180695.14it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


- 띄어쓰기 및 불용어 처리

In [ ]:
# 한국어 불용어 리스트 크롤링


url = "https://www.ranks.nl/stopwords/korean"
response = requests.get(url, verify = False)

if response.status_code == 200:
    soup = BeautifulSoup(response.text,'html.parser')
    content = soup.select_one('#article178ebefbfb1b165454ec9f168f545239 > div.panel-body > table > tbody > tr')
    stop_words=[]
    for x in content.strings:
        x=x.strip()
        if x:
            stop_words.append(x)
    print(f"# Korean stop words: {len(stop_words)}")
else:
    print(response.status_code)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ranks.nl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


# Korean stop words: 677


In [ ]:
okt = Okt()

def title_tokenizing(x):
    temp_data = okt.morphs(x)
    temp_list = []
    for word in temp_data:
        if word in stop_words: 
            continue
        temp_list.append(word)
  
    return " ".join(temp_list)
    
    
def content_tokenizing(x):
    new_list = list(filter(None, x))
    
    final_list = []
    for i in new_list:
        temp_data = okt.morphs(i)
        temp_list = []
        for word in temp_data:
            if word in stop_words:
                continue
            temp_list.append(word)
        final_list.append(" ".join(temp_list))

    return final_list

In [ ]:
df_new['tokenized_TITLE'] = df_new['TITLE'].progress_apply(lambda x: title_tokenizing(x))
df_new['tokenized_CONTENT'] = df_new['listed_CONTENT'].progress_apply(lambda x: content_tokenizing(x))

100%|██████████| 8998/8998 [00:14<00:00, 604.26it/s] 
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
100%|██████████| 8998/8998 [04:30<00:00, 33.29it/s]


In [ ]:
df_new = df_new[['tokenized_TITLE', 'tokenized_CONTENT']]
df_new.rename(columns={'tokenized_CONTENT': 'CONTENT', 'tokenized_TITLE':'TITLE'}, inplace = True)
df_new

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,TITLE,CONTENT
0,스털링 다이빙 논란 종결 오른쪽 다리 접촉 있었잖아,[유럽 축구 연맹 유로 2020 심판 위원장 로베르토 로세티 잉글랜드 덴마크 전 나...
1,디 마리아 없다 유로 X 코파 베스트 11 이탈리아 만 7 명,"[유로 2020 코파 아메리카 2021 베스트 11 만든다면 모습 일까, 지난달 시..."
2,슈퍼컴퓨터 예측 맨시티 우승 맨유 4 위 토트넘 은 6 위,"[새 시즌 시작 하기도 전 슈퍼컴퓨터 예상 한 순위 나왔다, 영국 매체 스포츠 바이..."
3,이재성 완벽한 프로 마인츠 서 성공할 킬 디렉터 애정 듬뿍 응원,"[홀슈타인 킬 우베 스토 버 디렉터 이재성 향 해 응원 메시지 띄웠다, 이재성 은 ..."
4,홈킷 딴판 바르사 팬 NEW 웨이 셔츠 호평 가장 좋아하는 색,"[FC 바르셀로나 새 시즌 원정 유니폼 공개 했다, 팬 은 만족스럽다는 반응 이다,..."
...,...,...
8993,이제 홈팬 야유 받는 먹튀 선수 차비 감독 그만 해 라,"[FC 바르셀로나 팬 우스 망 뎀벨레 는 밉 상이 되어 버렸다, 바르사 는 7일 오..."
8994,성남 만 17 세 유스 김지수 준 프로 계약,"[성남 FC 만 17 세 2004년 생 수비수 김지수 준 프로 계약 체결 했다, 김..."
8995,오베르마스 아약스 서 쫓겨난다 ... 여성 동료 부적절한 메시지,"[레전드 마르크 오베르마스 아약스 쫓겨났다, 이유 는 굉장히 굴욕 적, 아약스 는 ..."
8996,바르사 차비 감독 트라오레 데뷔전 활약 깜짝 ... 몸 야수 같아,"[FC 바르셀로나 차비 에르난데스 감독 데뷔전 치른 다마 트라오레 극찬 했다, 바르..."


In [ ]:
df_new.loc[0, 'CONTENT']

['유럽 축구 연맹 유로 2020 심판 위원장 로베르토 로세티 잉글랜드 덴마크 전 나온 판정 논란 정심 이라고 공언 했다',
 '지난 8일 잉글랜드 덴마크 는 유로 2020 4 강 결승 티켓 두고 격돌 했다',
 '연장 접전 끝 잉글랜드 21 이겼다',
 '경기 후 논란 불거졌다',
 '11 팽팽 하던 연장 전반 12분 라 힘 스털링 드리블 돌파 하던 중 요아킴 멜레 마티아스 옌센 사이 넘어졌다',
 '심판 은 곧장 페널티 스팟 찍었다',
 '비디오 판독 실과 의견 나눈 뒤 에도 원심 유지 했다',
 '페널티킥 얻은 잉글랜드 는 해리 케인 실축 했지만 흐른 볼 밀어 넣어 결승 티켓 따냈다',
 '장면 두고 갑론 박 펼쳐졌다',
 '스털링 은 경기 후 인터뷰 명백한 페널티킥 이라고 주장 했지만 전문가 의견 은 달랐다',
 '조제 모리뉴 AS 로마 감독 아르 센 벵거 전 아스널 감독 은 페널티킥 아니다고 입 모았다',
 '많은 이야기 흘러나오는 가운데 유로 2020 심판 위원장 로세티 오심 아니라는 입장 내놨다',
 '로세티 위원장 은 14일 영국 매체 가디언 인터뷰 주심 은 5 번 수비수 주목 했다',
 '수비수 볼 터치 하지 않았다고 봤다',
 '멜레 오른쪽 다리 스털링 오른쪽 다리 접촉 한 확인 했다',
 '접촉 강도 논 할 수 있지만 는 항상 심판 의사결정 과정 중심 되길 바란다고 밝혔다',
 '로세티 위원장 심판 대표 해 의견 냈지만 오심 이라고 생각 하는 받아들일지는 미지수 다',
 '스털링 은 평소 에도 다이빙 논란 시 달려왔고 많은 머릿속 다이버 라는 인식 가득하기 때문 이다']

In [ ]:
df_new['len'] = df_new['CONTENT'].apply(lambda x: len(x))
max(df_new['len'].to_list())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


249

#Extractive summarization - Matchsum

### Dataset & Dataloader 생성

In [ ]:
def control_input_ids(input_ids_tensor,length,cls_token_num,sep_token_num,pad_token_num):
  cur_length = len(input_ids_tensor)
  cls_token = torch.tensor([cls_token_num])
  sep_token = torch.tensor([sep_token_num])

  if cur_length+2 > length:
    input_ids_tensor = input_ids_tensor[:length-2]  # 길이가 넘치면 자른다
    return torch.cat([cls_token,input_ids_tensor,sep_token])
  else:
    input_ids_tensor = torch.cat([cls_token,input_ids_tensor,sep_token])
    padding_list = torch.tensor([pad_token_num]*(length - cur_length -2)) # 길이가 모자라면 padding token 을 채운다
    return torch.cat([input_ids_tensor,padding_list])

In [ ]:
def custom_collate_fn(samples):
  
  text_ids = torch.empty(0,512)
  labels_ids = torch.empty(0,32)
  for sample in samples:
    text_ids = torch.cat([text_ids,sample['text_input_ids'].unsqueeze(0)],dim=0) 
    labels_ids = torch.cat([labels_ids,sample['labels_input_ids'].unsqueeze(0)],dim=0)

  sentence_input_ids = [sample['sentence_input_ids'] for sample in samples]
  nn.utils.rnn.pad_sequence(sentence_input_ids,batch_first=True,padding_value = 1)

  return dict(text_input_ids = text_ids.to(torch.int64), labels_input_ids = labels_ids.to(torch.int64), sentence_input_ids = sentence_input_ids)

In [ ]:
class CustomDataset(Dataset):
  def __init__(
      self, data, tokenizer,
      text_max_token_len = 512,
      summary_max_token_len = 32
        ):
    self.tokenizer = tokenizer
    self.data = data
    self.text_max_token_len = text_max_token_len
    self.summary_max_token_len = summary_max_token_len
  def __len__(self):
    return len(self.data)
  
  def __getitem__(self, index):
    cls_token_num = 0
    sep_token_num = 2
    pad_token_num = 1
    
    data_row = self.data.iloc[index]
    text = data_row['CONTENT']
    
    total_text_ids = torch.tensor([])
    sentence_input_ids = torch.empty(0,32)

    for sentence in text:
      text_encoding_sentence = self.tokenizer(
          sentence,return_tensors = "pt",add_special_tokens=False)
      sentence_indiv_input_ids = text_encoding_sentence['input_ids'].flatten()
      total_text_ids = torch.cat([total_text_ids,sentence_indiv_input_ids])

      sentence_indiv_input_ids = control_input_ids(sentence_indiv_input_ids,self.summary_max_token_len,cls_token_num,sep_token_num,pad_token_num)
      sentence_indiv_input_ids = sentence_indiv_input_ids.unsqueeze(0)
      sentence_input_ids = torch.cat([sentence_input_ids,sentence_indiv_input_ids],dim=0)
    
    sentence_input_ids = sentence_input_ids.to(torch.int64)
    total_text_ids = control_input_ids(total_text_ids,self.text_max_token_len,cls_token_num,sep_token_num,pad_token_num)    
    total_text_ids = total_text_ids

    labels = data_row['TITLE']
    summary_encoding = self.tokenizer(
        labels,
        add_special_tokens = False,
        return_tensors = "pt"
    )

    labels_ids = summary_encoding['input_ids'].flatten()
    labels_ids = control_input_ids(labels_ids,self.summary_max_token_len,cls_token_num,sep_token_num,pad_token_num)

    return dict(text_input_ids = total_text_ids, labels_input_ids = labels_ids, sentence_input_ids = sentence_input_ids)


In [ ]:
tokenizer = get_kobart_tokenizer()

/content/.cache/kobart_base_tokenizer_cased_cf74400bce.zip[██████████████████████████████████████████████████]


In [ ]:
whole_dataset = CustomDataset(df_new,tokenizer)

train_set_num = len(df_new)//9*7
train_dataset , valid_dataset = random_split(whole_dataset, [train_set_num,len(df_new)-train_set_num])
train_dataloader = DataLoader(train_dataset, batch_size = 2, shuffle=True,collate_fn = custom_collate_fn)
valid_dataloader =  DataLoader(valid_dataset, batch_size = 2, shuffle=False,collate_fn = custom_collate_fn)

In [ ]:
next(iter(train_dataloader))

{'labels_input_ids': tensor([[    0, 14086, 15506, 23517, 14806, 16276, 26869, 13469, 14137, 12037,
          27578, 14402, 20007, 14082,     2,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1],
         [    0, 14025,  9000, 12037, 14683,  9776, 16959,  9866, 14347, 13455,
          15154, 15464,   274, 15819, 12332, 14710, 11715,  9754,     2,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1]]),
 'sentence_input_ids': [tensor([[    0, 15251, 16449, 14091, 10527, 27890, 16400, 14272, 11986, 14154,
           23517, 16211, 14669, 14036, 14591, 27578, 14402, 23980, 14784, 14307,
           11440, 20007, 14082, 14497,  9766, 15988, 29365,     2,     1,     1,
               1,     1],
          [    0, 16649,   268, 22758, 15494, 16625, 25477, 16513, 11857, 15658,
           14036, 14091, 10527, 21596, 19250,   248, 14745, 21345, 16649,   

### Matchsum

- 평가 metric -> rdass
- 기본적으로 모델에 스코어가 높은 5개의 단일 문장을 뽑고 뽑인 문장으로 만들어진 조합 가운데서 스코어가 높은 조합을 golden summary로 선정
- loss 는 margin ranking loss 사용


In [48]:
def get_score(doc,label,answer):
  score_1 = torch.cosine_similarity(doc,answer,dim=0)
  score_2 = torch.cosine_similarity(label,answer,dim=0)
  return (score_1+score_2)/2

In [14]:
def get_candidate_id(doc_emb,summary_emb,batch_sentence_id, candidate_num, extract_model,device):
    cls_token = torch.tensor([0]).to(device)
    sep_token = torch.tensor([2]).to(device)
    candidate_ids = torch.empty([0,candidate_num,128]).to(device)
    
    for batch_idx, sentence_id_tensor in enumerate(batch_sentence_id):
      sentence_id_tensor = sentence_id_tensor.to(device)
      out = extract_model.forward(sentence_id_tensor)  #sentence_id_tensor = [문장 갯수,32개의 토큰]
      hidden_states = out['last_hidden_state'][:,0,:] # [문장 갯수,token 갯수 ,768 dim_vec]
      score_list= []
      
      for i in range(hidden_states.shape[0]):
        score = get_score(doc = doc_emb[batch_idx,:], label = summary_emb[batch_idx,:], answer = hidden_states[i,:])
        score_list.append((score,i))
      
      score_list.sort(key = lambda x: x[0],reverse=True)
      idx_list = [idx for _,idx in score_list][:5]
    
      # get candidate summaries
      # here is for CNN/DM: truncate each document into the 5 most important sentences (using BertExt), 
      # then select any 2 or 3 sentences to form a candidate summary, so there are C(5,2)+C(5,3)=20 candidate summaries.
      # if you want to process other datasets, you may need to adjust these numbers according to specific situation.
      indices = list(combinations(idx_list, 2))
      indices += list(combinations(idx_list, 3))
      
      if len(indices) < 2:
        indices = [idx_list]
      
      len_indices = len(indices) 
      if len_indices < candidate_num : 
        indices = indices*(candidate_num//len_indices)
        indices.append(idx_list[:-(candidate_num%len_indices)])

      # get score for each candidate summary and sort them in descending order
      score = []
      for i in indices:
          i = list(i)
          i.sort()
          # write dec
          dec = torch.tensor([]).to(device)
          for j in i:
              sent = sentence_id_tensor[j]
              sent = sent[1:]
              sep_token_idx = 0
              for token_idx in range(len(sent)):
                if sent[token_idx] == 2: break
                else:sep_token_idx += 1
              sent = sent[:sep_token_idx]
              dec = torch.cat([dec,sent],dim=0)
          
          dec = torch.cat([cls_token,dec,sep_token],dim=0)
          dec = dec.to(torch.int64)
          dec_out = extract_model.forward(input_ids = dec.unsqueeze(0))
          score.append((dec, get_score(doc_emb[batch_idx,:],summary_emb[batch_idx,:], dec_out['last_hidden_state'][0,0,:])))
      
      score.sort(key=lambda x : x[1], reverse=True)
      score = score[:candidate_num]
      
      candidate_ids_ind= torch.empty(0,128).to(device)
      for k,_ in score:
        dec = k
        if len(dec) < 128:
          padding_list = torch.tensor([1]*(128-len(dec))).to(device)
          dec = torch.cat([k,padding_list],dim=0)
        else:
          dec = dec[:128]

        candidate_ids_ind = torch.cat([candidate_ids_ind,dec.unsqueeze(0)],dim = 0)

      candidate_ids = torch.cat([candidate_ids,candidate_ids_ind.unsqueeze(0)],dim = 0)

    return candidate_ids.to(torch.int64)

In [15]:
class MatchSum(nn.Module):  
    def __init__ (self, encoder, candidate_num, device,hidden_size=768):
        super(MatchSum, self).__init__()
        
        self.hidden_size = hidden_size
        self.candidate_num  = candidate_num
        self.encoder = encoder
        self.device = device

    def forward(self, text_id, summary_id,list_of_sentence_id):
        
        batch_size = text_id.size(0)
        pad_id = 1 

        # get document embedding
        input_mask = ~(text_id == pad_id)
        out = self.encoder(text_id, attention_mask=input_mask)['last_hidden_state'] # last layer
        doc_emb = out[:, 0, :]
        assert doc_emb.size() == (batch_size, self.hidden_size) # [batch_size, hidden_size]
        
        # get summary embedding
        input_mask = ~(summary_id == pad_id)
        out = self.encoder(summary_id, attention_mask=input_mask)['last_hidden_state'] # last layer
        summary_emb = out[:, 0, :]
        assert summary_emb.size() == (batch_size, self.hidden_size) # [batch_size, hidden_size]

        # get summary score
        summary_score = torch.cosine_similarity(summary_emb, doc_emb, dim=-1)

        # get candidate embedding
        candidate_id = get_candidate_id(doc_emb,summary_emb,list_of_sentence_id, self.candidate_num, self.encoder,self.device) #[batch_size , candidate_num, token_num]
        candidate_id_copy = candidate_id
        candidate_id = candidate_id.view(-1, candidate_id.size(-1)) 
        input_mask = ~(candidate_id == pad_id)
        out = self.encoder(candidate_id, attention_mask=input_mask)['last_hidden_state'] 
        candidate_emb = out[:, 0, :].view(batch_size, self.candidate_num, self.hidden_size)  # [batch_size, candidate_num, hidden_size]
        assert candidate_emb.size() == (batch_size, self.candidate_num, self.hidden_size)
        
        # get candidate score
        doc_emb = doc_emb.unsqueeze(1).expand_as(candidate_emb)
        score = torch.cosine_similarity(candidate_emb, doc_emb, dim=-1) # [batch_size, candidate_num]
        golden_list = torch.argmax(score,dim=1)
        assert score.size() == (batch_size, self.candidate_num)

        candidate_id = candidate_id.view(-1, candidate_id.size(-1)) 

        return {'score': score, 'summary_score': summary_score,  'golden_summary':candidate_id_copy[:,0,:]}

In [16]:
class MarginRankingLoss():      
    
    def __init__(self, margin, score=None, summary_score=None):
        super(MarginRankingLoss, self).__init__()
        # self._init_param_map(score=score, summary_score=summary_score)
        self.margin = margin
        self.loss_func = torch.nn.MarginRankingLoss(margin)

    def get_loss(self, score, summary_score):
        
        # equivalent to initializing TotalLoss to 0
        # here is to avoid that some special samples will not go into the following for loop
        ones = torch.ones(score.size()).cuda(score.device)
        loss_func = torch.nn.MarginRankingLoss(0.0)
        TotalLoss = loss_func(score, score, ones)

        # candidate loss
        n = score.size(1)
        for i in range(1, n):
            pos_score = score[:, :-i]
            neg_score = score[:, i:]
            pos_score = pos_score.contiguous().view(-1)
            neg_score = neg_score.contiguous().view(-1)
            ones = torch.ones(pos_score.size()).cuda(score.device)
            loss_func = torch.nn.MarginRankingLoss(self.margin * i)
            TotalLoss += loss_func(pos_score, neg_score, ones)

        # gold summary loss
        pos_score = summary_score.unsqueeze(-1).expand_as(score)
        neg_score = score
        pos_score = pos_score.contiguous().view(-1)
        neg_score = neg_score.contiguous().view(-1)
        ones = torch.ones(pos_score.size()).cuda(score.device)
        loss_func = torch.nn.MarginRankingLoss(0.0)
        TotalLoss += loss_func(pos_score, neg_score, ones)
        
        return TotalLoss

### Train code

- Encoder -> KoBART
- GLM 을 제외한 제일 성능 좋은 모델이고 한국어로 train이 되어 있어 선정함

In [ ]:
model = BartModel.from_pretrained(get_pytorch_kobart_model())
summary_model = MatchSum(encoder = model, candidate_num = 5,device = device, hidden_size=768) 

N_EPOCHS = 3
optimizer = SGD(model.parameters(),lr =0.0001)
scheduler = CosineAnnealingLR(optimizer,T_max = len(train_dataloader)*2)
criterion = MarginRankingLoss(margin = 0.01)

In [ ]:
model.to(device)
summary_model.to(device)
wandb.init(project='summarization', entity='tkdlqh2')

for epoch in range(N_EPOCHS):
    
    print(f"*****Epoch {epoch} Train Start*****")
    print(f"*****Epoch {epoch} Total Step {len(train_dataloader)}*****")
    total_loss, batch_loss, batch_step = 0,0,0
    model.train()

    for step, batch in enumerate(train_dataloader):
        batch_step+=1
        text_input_ids = batch["text_input_ids"].to(device)        
        label_input_ids = batch["labels_input_ids"].to(device)

        model.zero_grad()
        optimizer.zero_grad()

        # forward
        output = summary_model.forward(text_input_ids, label_input_ids,batch["sentence_input_ids"])
        loss = criterion.get_loss(score = output["score"],summary_score = output["summary_score"])

        # loss 계산
        loss.backward()
        # optimizer 업데이트
        optimizer.step()
        # scheduler 업데이트
        scheduler.step()

        batch_loss += loss.item()
        total_loss += loss.item()

        learning_rate = optimizer.param_groups[0]['lr']
        wandb.log({'train/lr':learning_rate,"train/loss":loss.item()})

        if (step%50 == 0) and (step!=0):
            print(f"Step: {step} Loss: {batch_loss/batch_step:.4f} lr: {optimizer.param_groups[0]['lr']:.4f}")
            # 변수 초기화    
            batch_loss, batch_step = 0,0

    print(f"Epoch {epoch} Total Mean Loss : {total_loss/(step+1):.4f}")
    
    with torch.no_grad():
      print('**Calculating validation results...**')
      total_val_loss,batch_step = 0,0
      model.eval()
      for step, batch in enumerate(valid_dataloader):
          batch_step+=1
          text_input_ids = batch["text_input_ids"].to(device)        
          label_input_ids = batch["labels_input_ids"].to(device)

          # forward
          output = summary_model.forward(text_input_ids, label_input_ids,batch["sentence_input_ids"])
          val_loss = criterion.get_loss(score = output["score"],summary_score = output["summary_score"])

          total_val_loss += val_loss.item()
          wandb.log({"val/loss":val_loss.item()})

    print(f"Epoch {epoch} Total Mean Score : {total_val_loss/(step+1):.4f}")
    
    print(f"*****Epoch {epoch} Train Finished*****\n")
    torch.save(model.state_dict(),f"/content/drive/MyDrive/NLP/kobart_model_{epoch}epoch.pth")

### Model-Load & Inference
- 

In [17]:
model = BartModel.from_pretrained(get_pytorch_kobart_model())
model.load_state_dict(state_dict = torch.load("/content/drive/MyDrive/NLP/kobart_model_0epoch.pth"))
model.eval()

/content/.cache/kobart_base_cased_ff4bda5738.zip[██████████████████████████████████████████████████]


KeyboardInterrupt: ignored

In [ ]:
model.to(device)
summary_model = MatchSum(encoder = model, candidate_num = 1,device = device, hidden_size=768) 

In [ ]:
def to_total(x):
  total_str = ""
  for sentence in x:
    total_str += " "
    total_str += sentence
  
  return total_str

class get_ids():
    def __init__(self,tokenizer):
      self.tokenizer = tokenizer

    def __call__(self, sentence):
      ids = self.tokenizer(sentence)['input_ids']
      ids = torch.tensor(ids)
      return control_input_ids(ids,32,0,2,1).unsqueeze(0).to(torch.int64)

class get_list_ids():
    def __init__(self,tokenizer,device):
      self.tokenizer = tokenizer
      self.device = device
    def __call__(self, sentence_list):

      ans = torch.empty((0,32),dtype = torch.int64).to(self.device)
      for sentence in sentence_list:
        ids = self.tokenizer(sentence)['input_ids']
        ids = torch.tensor(ids,dtype = torch.int64)
        ans = torch.cat([ans,control_input_ids(ids,32,0,2,1).unsqueeze(0).to(torch.int64).to(device)],dim=0)
      return [ans]

In [ ]:
ids_class = get_ids(tokenizer)
ids_list_class = get_list_ids(tokenizer,device)

In [ ]:
df_new["text"] = df_new["CONTENT"].apply(lambda x: to_total(x))
df_new["text_ids"] = df_new["text"].apply(lambda x: ids_class(x))
df_new["label_ids"] = df_new["TITLE"].apply(lambda x: ids_class(x))
df_new['list_of_sentence_ids'] = df_new["CONTENT"].progress_apply(lambda x: ids_list_class(x))

In [ ]:
df_new['model_answer'] = df_new["CONTENT"]
for i in tqdm(range(len(df_new))):
   output = summary_model.forward(df_new["text_ids"].iloc[i].to(device),df_new["label_ids"][i].to(device),df_new['list_of_sentence_ids'].iloc[i])
   df_new['model_answer'].iloc[i] = output['golden_summary'].detach().cpu().numpy()

In [ ]:
df_new['model_answer_sentence'] = df_new['model_answer'].apply(lambda x: tokenizer.decode(token_ids = x[0], skip_special_tokens = True))

In [ ]:
print(tokenizer.decode(token_ids = [1]))


In [ ]:
df_new_for_answer = df_new[['TITLE','CONTENT','model_answer_sentence']]
df_new_for_answer.to_csv("/content/drive/MyDrive/temp_df.csv",mode = 'w')

In [ ]:
idx = 7
print(df_new_for_answer['TITLE'].iloc[idx])
print(df_new_for_answer['CONTENT'].iloc[idx])
print(df_new_for_answer['model_answer_sentence'].iloc[idx])

In [20]:
df_new_for_answer = pd.read_csv("/content/drive/MyDrive/temp_df.csv")

In [49]:
class rdass:
  def __init__(self,encoder,tokenizer):
    self.encoder = encoder
    self.tokenizer = tokenizer
    self.encoder.eval()
  def __call__(self, text = None, label = None, answer = None):
    with torch.no_grad():
      text_ids = self.tokenizer(text,truncation = True,max_length = 512,return_tensors = 'pt')
      label_ids = self.tokenizer(label,return_tensors = 'pt')
      answer_ids = self.tokenizer(answer,return_tensors = 'pt')
      # print(text_ids['input_ids'].shape," ",label_ids['input_ids'].shape," ",answer_ids['input_ids'].shape)
      vector_text = self.encoder(**text_ids)['last_hidden_state'] # vector_d
      vector_label = self.encoder(**label_ids)['last_hidden_state'] # vector_r
      vector_answer = self.encoder(**answer_ids)['last_hidden_state'] # vector_p
      
    return get_score(vector_text[0,0,:],vector_label[0,0,:],vector_answer[0,0,:])

In [22]:
tokenzier_for_eval = AutoTokenizer.from_pretrained("klue/roberta-small")
model_for_eval = AutoModel.from_pretrained("klue/roberta-small")
metric = rdass(model_for_eval,tokenzier_for_eval)

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [50]:
metric = rdass(model_for_eval,tokenzier_for_eval)

In [33]:
print(df_new_for_answer['model_answer_sentence'])

0       유럽 축구 연맹 유로 2020 심판 위원장 로베르토 로세티 잉글랜드 덴마크 전 나온...
1       유로 2020 코파 아메리카 2021 베스트 11 만든다면 모습 일까 공식 베스트 ...
2       영국 매체 스포츠 바이블 은 14일 슈퍼컴퓨터 예측 한 20212022시 즌 잉글랜...
3       홀슈타인 킬 우베 스토 버 디렉터 이재성 향 해 응원 메시지 띄웠다 팬 만남 기대 ...
4       팬 은 만족스럽다는 반응 이다 연 보랏빛 바탕 메인 스폰서 라쿠텐 은 파란색 새겨져...
                              ...                        
8993    FC 바르셀로나 팬 우스 망 뎀벨레 는 밉 상이 되어 버렸다 바르사 는 7일 오전 ...
8994    U 15 크로아티아 국제 축구 대회 AFC U 16 챔피언십 예선 참여 2021년 ...
8995    아약스 는 7일 공식 홈페이지 통해 기술 이사 인 오베르마스 는 아약스 떠난다 오베...
8996    트라오레 는 지난 겨울 이적 시장 울버햄프턴 떠나 바르사 임대 이적 했고 날 경기 ...
8997    공 격진 에는 멤피스 데 파이 라다 멜 팔카오 알 렉시스 산체스 포함 됐다 이외 프...
Name: model_answer_sentence, Length: 8998, dtype: object


In [51]:
df_new_for_answer['score'] = df_new_for_answer['CONTENT']
for i in tqdm(range(len(df_new_for_answer))):
  df_new_for_answer['score'].iloc[i] = metric(
                                            text = df_new_for_answer['CONTENT'].iloc[i],
                                            label = df_new_for_answer['TITLE'].iloc[i], 
                                            answer = df_new_for_answer['model_answer_sentence'].iloc[i]
  )

  0%|          | 0/8998 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
100%|██████████| 8998/8998 [52:21<00:00,  2.86it/s]


In [52]:
df_new_for_answer.head(20)

,Unnamed: 0,TITLE,CONTENT,model_answer_sentence,score
0,0,스털링 다이빙 논란 종결 오른쪽 다리 접촉 있었잖아,['유럽 축구 연맹 유로 2020 심판 위원장 로베르토 로세티 잉글랜드 덴마크 전 ...,유럽 축구 연맹 유로 2020 심판 위원장 로베르토 로세티 잉글랜드 덴마크 전 나온...,tensor(0.9180)
1,1,디 마리아 없다 유로 X 코파 베스트 11 이탈리아 만 7 명,"['유로 2020 코파 아메리카 2021 베스트 11 만든다면 모습 일까', '지난...",유로 2020 코파 아메리카 2021 베스트 11 만든다면 모습 일까 공식 베스트 ...,tensor(0.9138)
2,2,슈퍼컴퓨터 예측 맨시티 우승 맨유 4 위 토트넘 은 6 위,"['새 시즌 시작 하기도 전 슈퍼컴퓨터 예상 한 순위 나왔다', '영국 매체 스포츠...",영국 매체 스포츠 바이블 은 14일 슈퍼컴퓨터 예측 한 20212022시 즌 잉글랜...,tensor(0.9219)
3,3,이재성 완벽한 프로 마인츠 서 성공할 킬 디렉터 애정 듬뿍 응원,"['홀슈타인 킬 우베 스토 버 디렉터 이재성 향 해 응원 메시지 띄웠다', '이재성...",홀슈타인 킬 우베 스토 버 디렉터 이재성 향 해 응원 메시지 띄웠다 팬 만남 기대 ...,tensor(0.9121)
4,4,홈킷 딴판 바르사 팬 NEW 웨이 셔츠 호평 가장 좋아하는 색,"['FC 바르셀로나 새 시즌 원정 유니폼 공개 했다', '팬 은 만족스럽다는 반응 ...",팬 은 만족스럽다는 반응 이다 연 보랏빛 바탕 메인 스폰서 라쿠텐 은 파란색 새겨져...,tensor(0.9066)
5,5,긴급 수혈 된 바르사 NO . 9 1년 반 만에 떠난다 EPL 행 유력,"['FC 바르셀로나 는 새 시즌 앞두고 선수단 정리 한창 이다', '잉여 자원 은 ...",FC 바르셀로나 는 새 시즌 앞두고 선수단 정리 한창 이다 잉여 자원 은 과감 하게...,tensor(0.8983)
6,6,황의조 손흥민 소속사 손잡다 CAA Base 계약,['스포 탈 코리아 파리 황의조 한국 선수 로는 3 번 째 세계 적 인 스포츠 에이...,스포 탈 코리아 파리 황의조 한국 선수 로는 3 번 째 세계 적 인 스포츠 에이전트...,tensor(0.9137)
7,7,메시 종신 은 축복 스폰서 5년 더 보장 바르셀로나 함박 웃음,"['리오넬 메시 FC 바르셀로나 남는다', '연봉 절반 삭감 5년 계약 사실 상 바...",연봉 절반 삭감 5년 계약 사실 상 바르셀로나 은퇴 할 결심 했다 메시 계약 소식 ...,tensor(0.9271)
8,8,눈물 흘렸던 37 세 전설 로번 두 번째 현역 은퇴 발표,"['네덜란드 축구 스타 아르 연 로번 현역 은퇴 밝혔다', '로번 은 15일 회관 ...",시즌 막바지 인 지난 5월 스파 르트 로테르담 전 통해 돌아왔고 이후 FC 에먼 전...,tensor(0.9200)
9,9,100 세 메시 팬 할아버지 748 골 수기 작 성 메시 도 감사 인사,"['리오넬 메시 는 프로 데뷔 하고 748 골 터뜨렸다', '전산 화 하기에도 오랜...",메시 사랑 하는 100 세 아르헨티나 할아버지 화제 다 손자 는 메시 득점 한 모든...,tensor(0.9339)
